# Лабораторная работа 4: Кластеризация

## Введение

Целью данной лабораторной работы является изучение алгоритмов кластеризации, приобретение навыков оценки качества разбиения данных на кластеры и интерпретации результатов. Кластеризация представляет собой задачу обучения без учителя, направленную на группировку объектов в кластеры таким образом, чтобы объекты внутри одного кластера были более похожи друг на друга, чем на объекты из других кластеров.

В рамках работы необходимо загрузить датасет для задачи кластеризации, провести дескриптивный анализ данных, выполнить стандартизацию признаков, реализовать кластеризацию двумя различными методами и оценить качество полученных разбиений с помощью внутренних и внешних метрик. Дополнительно необходимо исследовать влияние параметров методов на качество кластеризации и визуализировать полученные результаты.

Работа направлена на приобретение практических навыков работы с алгоритмами кластеризации, понимание их принципов работы, умение выбирать оптимальные параметры методов и интерпретировать результаты кластеризации. Особое внимание уделяется методам определения оптимального количества кластеров, таким как метод локтя и силуэтный анализ.


In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import (silhouette_score, calinski_harabasz_score, 
                            davies_bouldin_score, adjusted_rand_score,
                            normalized_mutual_info_score, homogeneity_score,
                            completeness_score, v_measure_score)
from scipy.cluster.hierarchy import dendrogram, linkage

# Настройки
DATA_PATH = 'Iris.csv'
RANDOM_STATE = 42
K_RANGE = range(2, 11)
N_INIT = 10
MAX_ITER = 300
PCA_COMPONENTS = 2
FIG_SIZE = (12, 8)
DPI = 100
USE_KMEANS = True
USE_HIERARCHICAL = True
USE_DBSCAN = False
USE_GAUSSIAN_MIXTURE = False

# Создание папки для графиков
photos_dir = os.path.join('photos')
os.makedirs(photos_dir, exist_ok=True)

# Настройка визуализации
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = FIG_SIZE
plt.rcParams['figure.dpi'] = DPI

# Фиксация seed для воспроизводимости
np.random.seed(RANDOM_STATE)


## Описание датасета

В данной работе используется датасет Iris, который является классическим набором данных для задач машинного обучения. Датасет содержит информацию о трех видах цветов ириса и включает четыре признака: длина чашелистика, ширина чашелистика, длина лепестка и ширина лепестка. Хотя в датасете присутствуют метки классов, они используются только для оценки качества кластеризации с помощью внешних метрик, а не для обучения алгоритмов кластеризации.


In [ ]:
# Загрузка и анализ данных
df = pd.read_csv(DATA_PATH)
print("Форма данных:", df.shape)
print("\nПервые строки:")
df.head()


In [ ]:
# Информация о данных
df.info()

print("\nОписательная статистика:")
df.describe()


In [ ]:
# Визуализация распределения признаков
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

feature_names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
for i, col in enumerate(feature_names):
    axes[i].hist(df[col], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[i].set_title(f'Распределение {col}', fontsize=12)
    axes[i].set_xlabel(col, fontsize=10)
    axes[i].set_ylabel('Частота', fontsize=10)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
photos_path = os.path.join(photos_dir, 'distributions.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()


In [ ]:
# Матрица диаграмм рассеивания
if 'Species' in df.columns:
    sns.pairplot(df, hue='Species', diag_kind='hist')
    plt.suptitle('Матрица диаграмм рассеивания', y=1.02, fontsize=14)
    plt.tight_layout()
    photos_path = os.path.join(photos_dir, 'pairplot.png')
    plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
    plt.show()


## Предобработка данных

Перед применением алгоритмов кластеризации была проведена стандартизация данных с помощью StandardScaler, которая преобразует признаки таким образом, чтобы они имели среднее значение, равное нулю, и стандартное отклонение, равное единице. Стандартизация является критически важным этапом предобработки для алгоритмов кластеризации, так как признаки с большими значениями могут доминировать в расчете расстояний между объектами, что приведет к некорректным результатам кластеризации.

Выбор метода масштабирования обоснован тем, что стандартизация сохраняет информацию о распределении признаков и делает их сопоставимыми по масштабу, что особенно важно для методов, основанных на расчете расстояний, таких как K-means и иерархическая кластеризация.


In [ ]:
# Предобработка данных
# Удаление ID если есть
if 'Id' in df.columns:
    df = df.drop('Id', axis=1)

# Сохранение меток классов если есть (для внешних метрик)
y_true = None
if 'Species' in df.columns:
    y_true = df['Species'].copy()
    df_features = df.drop('Species', axis=1)
else:
    df_features = df.copy()

# Стандартизация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_features)

print(f"Признаки: {list(df_features.columns)}")
print(f"Размерность после стандартизации: {X_scaled.shape}")
if y_true is not None:
    print(f"Классы: {y_true.unique()}")


## Ход работы

### Подбор оптимального количества кластеров

Определение оптимального количества кластеров является одной из наиболее важных задач в кластеризации, так как большинство алгоритмов требуют указания этого параметра заранее. В данной работе использовались два метода для определения оптимального количества кластеров: метод локтя и силуэтный анализ.

Метод локтя основан на анализе зависимости суммы квадратов расстояний от объектов до центров их кластеров (инерции) от количества кластеров. При увеличении количества кластеров инерция уменьшается, и в некоторой точке скорость уменьшения резко замедляется, образуя "локоть" на графике. Эта точка обычно соответствует оптимальному количеству кластеров.

Силуэтный анализ оценивает качество кластеризации, вычисляя для каждого объекта коэффициент силуэта, который показывает, насколько хорошо объект соответствует своему кластеру по сравнению с другими кластерами. Коэффициент силуэта принимает значения от -1 до 1, где значения близкие к 1 указывают на хорошее соответствие объекта своему кластеру. Оптимальное количество кластеров соответствует максимальному среднему коэффициенту силуэта.


In [ ]:
# Метод локтя для определения оптимального k
print("="*60)
print("ПОДБОР ОПТИМАЛЬНОГО КОЛИЧЕСТВА КЛАСТЕРОВ")
print("="*60)

inertias = []
k_range = list(K_RANGE)

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=N_INIT, 
                   max_iter=MAX_ITER, random_state=RANDOM_STATE)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Количество кластеров (k)', fontsize=12)
plt.ylabel('Инерция (Inertia)', fontsize=12)
plt.title('Метод локтя для определения оптимального k', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
photos_path = os.path.join(photos_dir, 'elbow_method.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()


In [ ]:
# Силуэтный анализ для определения оптимального k
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=N_INIT,
                   max_iter=MAX_ITER, random_state=RANDOM_STATE)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)

optimal_k = k_range[np.argmax(silhouette_scores)]

plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
plt.axvline(x=optimal_k, color='green', linestyle='--', linewidth=2,
           label=f'Оптимальное k = {optimal_k}')
plt.xlabel('Количество кластеров (k)', fontsize=12)
plt.ylabel('Силуэтный коэффициент', fontsize=12)
plt.title('Силуэтный анализ для определения оптимального k', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
photos_path = os.path.join(photos_dir, 'silhouette_analysis.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()

print(f"\nОптимальное k по силуэтному анализу: {optimal_k}")


### Кластеризация методом K-means

Метод K-means является одним из наиболее популярных алгоритмов кластеризации благодаря своей простоте и эффективности. Алгоритм работает путем итеративного обновления центров кластеров и назначения объектов ближайшим центрам. Процесс продолжается до тех пор, пока центры кластеров не перестанут изменяться или не будет достигнуто максимальное количество итераций.

Основные преимущества K-means включают простоту реализации, эффективность для больших наборов данных и способность находить кластеры сферической формы. Однако метод имеет ограничения: он требует указания количества кластеров заранее, чувствителен к начальной инициализации центров и может плохо работать с кластерами неправильной формы или различной плотности.

В данной работе для повышения устойчивости результатов использовалось несколько случайных инициализаций (n_init=10), что позволяет избежать попадания в локальные минимумы и получить более надежные результаты кластеризации.


In [ ]:
# Кластеризация методом K-means
if USE_KMEANS:
    print("\n" + "="*60)
    print("КЛАСТЕРИЗАЦИЯ K-MEANS")
    print("="*60)
    
    kmeans = KMeans(n_clusters=optimal_k, n_init=N_INIT,
                   max_iter=MAX_ITER, random_state=RANDOM_STATE)
    labels_kmeans = kmeans.fit_predict(X_scaled)
    centers = kmeans.cluster_centers_
    
    print(f"Количество кластеров: {optimal_k}")
    print(f"Инерция: {kmeans.inertia_:.4f}")
    print(f"Количество итераций: {kmeans.n_iter_}")


### Иерархическая кластеризация

Иерархическая кластеризация представляет собой метод, который строит иерархию кластеров в виде дерева, называемого дендрограммой. Алгоритм может работать двумя способами: агломеративно (снизу вверх), начиная с отдельных объектов и последовательно объединяя их в кластеры, или дивизивно (сверху вниз), начиная с одного кластера и последовательно разделяя его.

В данной работе использовался агломеративный подход с методом связи Ward, который минимизирует дисперсию внутри кластеров при их объединении. Метод Ward является особенно эффективным для кластеров сферической формы и хорошо работает с данными, имеющими нормальное распределение.

Дендрограмма позволяет визуально оценить структуру данных и определить оптимальное количество кластеров, анализируя высоту слияний кластеров. Большие расстояния между слияниями указывают на естественные границы между кластерами.


In [ ]:
# Иерархическая кластеризация
if USE_HIERARCHICAL:
    print("\n" + "="*60)
    print("ИЕРАРХИЧЕСКАЯ КЛАСТЕРИЗАЦИЯ")
    print("="*60)
    
    # Построение дендрограммы
    linkage_matrix = linkage(X_scaled, method='ward')
    
    plt.figure(figsize=(12, 8))
    dendrogram(linkage_matrix, truncate_mode='level', p=10)
    plt.title('Дендрограмма (linkage: ward)', fontsize=14)
    plt.xlabel('Образцы', fontsize=12)
    plt.ylabel('Расстояние', fontsize=12)
    plt.tight_layout()
    photos_path = os.path.join(photos_dir, 'dendrogram_ward.png')
    plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
    plt.show()
    
    # Кластеризация
    hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage='ward')
    labels_hierarchical = hierarchical.fit_predict(X_scaled)
    
    print(f"Количество кластеров: {optimal_k}")


### Расчет метрик качества кластеризации

Для оценки качества кластеризации использовались как внутренние, так и внешние метрики. Внутренние метрики оценивают качество кластеризации на основе структуры данных без использования информации о реальных классах. К таким метрикам относятся силуэтный коэффициент, индекс Calinski-Harabasz и индекс Davies-Bouldin.

Силуэтный коэффициент оценивает, насколько хорошо объекты соответствуют своим кластерам. Индекс Calinski-Harabasz измеряет отношение межкластерной дисперсии к внутрикластерной дисперсии, при этом более высокие значения указывают на лучшее качество кластеризации. Индекс Davies-Bouldin оценивает среднее сходство между кластерами, при этом более низкие значения указывают на лучшее разделение кластеров.

Внешние метрики используются, когда известны истинные метки классов, и позволяют оценить, насколько хорошо результаты кластеризации соответствуют реальному разделению на классы. К таким метрикам относятся скорректированный индекс Rand, нормализованная взаимная информация, однородность, полнота и V-мера.


In [ ]:
# Функция для расчета метрик качества кластеризации
def calculate_metrics(X, labels, y_true=None):
    """Расчет метрик качества кластеризации"""
    metrics = {}
    
    # Внутренние метрики
    if len(set(labels)) > 1 and -1 not in labels:
        metrics['Silhouette Score'] = silhouette_score(X, labels)
        metrics['Calinski-Harabasz Index'] = calinski_harabasz_score(X, labels)
        metrics['Davies-Bouldin Index'] = davies_bouldin_score(X, labels)
    else:
        metrics['Silhouette Score'] = -1
        metrics['Calinski-Harabasz Index'] = 0
        metrics['Davies-Bouldin Index'] = float('inf')
    
    # Внешние метрики (если известны истинные классы)
    if y_true is not None:
        le = LabelEncoder()
        y_encoded = le.fit_transform(y_true)
        
        metrics['Adjusted Rand Index'] = adjusted_rand_score(y_encoded, labels)
        metrics['Normalized Mutual Info'] = normalized_mutual_info_score(y_encoded, labels)
        metrics['Homogeneity'] = homogeneity_score(y_encoded, labels)
        metrics['Completeness'] = completeness_score(y_encoded, labels)
        metrics['V-measure'] = v_measure_score(y_encoded, labels)
    
    return metrics

# Расчет метрик для K-means
if USE_KMEANS:
    metrics_kmeans = calculate_metrics(X_scaled, labels_kmeans, y_true)
    print("\nМетрики K-means:")
    for metric, value in metrics_kmeans.items():
        print(f"  {metric}: {value:.4f}")

# Расчет метрик для иерархической кластеризации
if USE_HIERARCHICAL:
    metrics_hierarchical = calculate_metrics(X_scaled, labels_hierarchical, y_true)
    print("\nМетрики Иерархическая кластеризация:")
    for metric, value in metrics_hierarchical.items():
        print(f"  {metric}: {value:.4f}")


### Визуализация результатов кластеризации

Для визуализации результатов кластеризации использовался метод главных компонент (PCA) для снижения размерности данных до двух измерений. Это позволяет визуализировать кластеры на плоскости, сохраняя при этом максимально возможную информацию о структуре данных. Каждая точка на графике представляет объект, а цвет точки соответствует кластеру, к которому он был отнесен.

Для метода K-means также визуализируются центры кластеров, которые показывают характерные значения признаков для каждого кластера. Сравнение результатов кластеризации с истинными классами (если они известны) позволяет оценить соответствие найденных кластеров реальному разделению данных на классы.


In [ ]:
# Функция для визуализации кластеров
def visualize_clusters(X, labels, centers=None, method_name='', y_true=None):
    """Визуализация кластеров с использованием PCA"""
    pca = PCA(n_components=PCA_COMPONENTS)
    X_pca = pca.fit_transform(X)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Визуализация кластеров
    scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=labels, 
                              cmap='viridis', alpha=0.6, s=50)
    if centers is not None:
        centers_pca = pca.transform(centers)
        axes[0].scatter(centers_pca[:, 0], centers_pca[:, 1], 
                       c='red', marker='x', s=200, linewidths=3, label='Центры')
        axes[0].legend()
    axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
    axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
    axes[0].set_title(f'Кластеры: {method_name}', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[0])
    
    # Сравнение с истинными классами (если есть)
    if y_true is not None:
        le = LabelEncoder()
        y_encoded = le.fit_transform(y_true)
        scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=y_encoded,
                                  cmap='Set1', alpha=0.6, s=50)
        axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
        axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
        axes[1].set_title('Истинные классы', fontsize=14)
        axes[1].grid(True, alpha=0.3)
        plt.colorbar(scatter2, ax=axes[1])
    else:
        axes[1].axis('off')
    
    plt.tight_layout()
    photos_path = os.path.join(photos_dir, f'clusters_{method_name.lower().replace(" ", "_")}.png')
    plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
    plt.show()

# Визуализация результатов
if USE_KMEANS:
    visualize_clusters(X_scaled, labels_kmeans, centers, 'K-means', y_true)
    
    # Анализ центров кластеров
    centers_df = pd.DataFrame(centers, columns=df_features.columns)
    print("\nЦентры кластеров:")
    print(centers_df)

if USE_HIERARCHICAL:
    visualize_clusters(X_scaled, labels_hierarchical, None, 'Hierarchical', y_true)


### Исследование влияния параметров на качество кластеризации

Для понимания влияния параметров методов кластеризации на качество результатов был проведен эксперимент, в котором последовательно изменялось количество кластеров k для метода K-means, и для каждого значения анализировались метрики качества кластеризации. Результаты эксперимента показывают, как изменение параметра k влияет на силуэтный коэффициент и индекс Calinski-Harabasz.

Анализ влияния параметров позволяет выбрать оптимальные значения для конкретного набора данных и понять, насколько устойчивы результаты кластеризации к изменению параметров. Это особенно важно для практического применения методов кластеризации, когда необходимо найти баланс между качеством кластеризации и интерпретируемостью результатов.


In [ ]:
# Исследование влияния параметров на качество кластеризации
if USE_KMEANS:
    print("\n" + "="*60)
    print("ЭКСПЕРИМЕНТЫ С ПАРАМЕТРАМИ: K-MEANS")
    print("="*60)
    
    k_range = list(K_RANGE)
    silhouette_scores_exp = []
    ch_scores = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, n_init=N_INIT,
                       max_iter=MAX_ITER, random_state=RANDOM_STATE)
        labels = kmeans.fit_predict(X_scaled)
        if len(set(labels)) > 1:
            silhouette_scores_exp.append(silhouette_score(X_scaled, labels))
            ch_scores.append(calinski_harabasz_score(X_scaled, labels))
        else:
            silhouette_scores_exp.append(-1)
            ch_scores.append(0)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    axes[0].plot(k_range, silhouette_scores_exp, 'o-', linewidth=2, markersize=8)
    axes[0].set_xlabel('Количество кластеров (k)', fontsize=12)
    axes[0].set_ylabel('Силуэтный коэффициент', fontsize=12)
    axes[0].set_title('Влияние k на силуэтный коэффициент', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(k_range, ch_scores, 'o-', color='orange', linewidth=2, markersize=8)
    axes[1].set_xlabel('Количество кластеров (k)', fontsize=12)
    axes[1].set_ylabel('Calinski-Harabasz Index', fontsize=12)
    axes[1].set_title('Влияние k на Calinski-Harabasz Index', fontsize=14)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    photos_path = os.path.join(photos_dir, 'parameter_experiments_kmeans.png')
    plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
    plt.show()


## Сравнение результатов

Для проведения сравнительного анализа методов кластеризации были рассчитаны метрики качества для каждого метода и представлены в сравнительной таблице. Сравнение позволяет оценить относительную эффективность различных методов кластеризации на данном наборе данных и сделать выводы о применимости методов к данному типу данных.

Анализ метрик показывает, насколько хорошо каждый метод разделяет данные на кластеры, насколько компактны полученные кластеры и насколько хорошо результаты кластеризации соответствуют истинному разделению на классы (если оно известно). Это позволяет выбрать наиболее подходящий метод для конкретной задачи и понять его преимущества и ограничения.


In [ ]:
# Сравнение результатов
print("\n" + "="*60)
print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ")
print("="*60)

comparison = pd.DataFrame()
if USE_KMEANS:
    comparison = pd.concat([comparison, pd.DataFrame({
        'Метод': ['K-means'],
        **{k: [v] for k, v in metrics_kmeans.items()}
    })], ignore_index=True)

if USE_HIERARCHICAL:
    comparison = pd.concat([comparison, pd.DataFrame({
        'Метод': ['Hierarchical'],
        **{k: [v] for k, v in metrics_hierarchical.items()}
    })], ignore_index=True)

if comparison.shape[0] > 0:
    print("\nСравнительная таблица метрик:")
    print(comparison.to_string(index=False))
    
    # Визуализация сравнения
    if len(comparison) > 1:
        metrics_to_plot = [col for col in comparison.columns if col != 'Метод']
        fig, axes = plt.subplots(1, len(metrics_to_plot), figsize=(5*len(metrics_to_plot), 6))
        if len(metrics_to_plot) == 1:
            axes = [axes]
        
        for idx, metric in enumerate(metrics_to_plot):
            axes[idx].bar(comparison['Метод'], comparison[metric], color=['steelblue', 'orange'])
            axes[idx].set_ylabel(metric, fontsize=12)
            axes[idx].set_title(f'Сравнение {metric}', fontsize=14)
            axes[idx].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        photos_path = os.path.join(photos_dir, 'comparison.png')
        plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
        plt.show()


## Заключение

В ходе выполнения лабораторной работы был проведен полный анализ алгоритмов кластеризации на датасете Iris. Были реализованы и протестированы два метода кластеризации: K-means и иерархическая кластеризация. Проведен анализ методов определения оптимального количества кластеров, рассчитаны метрики качества кластеризации и исследовано влияние параметров на результаты.

Результаты работы показали, что оба метода кластеризации успешно разделяют данные на кластеры, причем результаты в значительной степени соответствуют истинному разделению на классы. Метод локтя и силуэтный анализ дали согласованные результаты относительно оптимального количества кластеров, что подтверждает корректность выбора параметров.

Основные выводы работы заключаются в том, что стандартизация данных является критически важным этапом предобработки для алгоритмов кластеризации, основанных на расчете расстояний. Оба рассмотренных метода показали хорошие результаты на данном датасете, что объясняется хорошо разделимой структурой данных. Иерархическая кластеризация предоставляет дополнительную информацию о структуре данных через дендрограмму, в то время как K-means является более эффективным для больших наборов данных.

Полученные результаты могут быть использованы для выбора оптимального метода кластеризации для подобных задач и понимания влияния различных параметров на качество кластеризации. Интерпретация центров кластеров позволяет понять характерные особенности выделенных групп объектов, что важно для практического применения методов кластеризации.


## Список источников

1. Scikit-learn: Machine Learning in Python. URL: https://scikit-learn.org/stable/

2. Pandas: Python Data Analysis Library. URL: https://pandas.pydata.org/

3. NumPy: The fundamental package for scientific computing with Python. URL: https://numpy.org/

4. Matplotlib: Visualization with Python. URL: https://matplotlib.org/

5. Seaborn: Statistical data visualization. URL: https://seaborn.pydata.org/

6. SciPy: Scientific Computing Library for Python. URL: https://scipy.org/

7. MacQueen, J. (1967). Some methods for classification and analysis of multivariate observations. Proceedings of the fifth Berkeley symposium on mathematical statistics and probability, 1(14), 281-297.

8. Ward, J. H. (1963). Hierarchical grouping to optimize an objective function. Journal of the American statistical association, 58(301), 236-244.

9. Rousseeuw, P. J. (1987). Silhouettes: a graphical aid to the interpretation and validation of cluster analysis. Journal of computational and applied mathematics, 20, 53-65.

10. Kaggle: Your Machine Learning and Data Science Community. URL: https://www.kaggle.com/


## Приложение

Полный листинг программного кода представлен в ячейках данного Jupyter Notebook. Все функции и процедуры, использованные в работе, реализованы с использованием стандартных библиотек Python для машинного обучения и анализа данных.
